## Installing libraries and dependencies

In [21]:
import openai
import pandas as pd
import json


In [22]:
# Set your OpenAI API key
openai.api_key = "sk-proj-AycHzZMxqZscz8ltuD5iT3BlbkFJvJPLk9TbP9cMwDCZJd2w"

## loading movies, users(training), interaction matrix, and test set

In [16]:
# path to the folder containing movielens data
Path = "D:/Canada/Danial/UoW/Dataset/MovieLens/100K/ml-100k"

movies_df = pd.read_pickle("./Data/movies_enriched_dataset.pkl")
movies_df = movies_df[["movie_id", "movie_info"]]

# loading users_df
users_df = pd.read_pickle("./Data/user_dataset.pkl")
users_df = users_df[["user_id", "user_info"]]

# Load the ratings data
rating_df = pd.read_csv(f'{Path}/u1.base', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'], encoding='latin-1')
rating_test_df = pd.read_csv(f'{Path}/u1.test', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'], encoding='latin-1')

In [20]:
users_df.head()

,user_id,user_info
0,1,\n Age: 24\n Gender: Male\n Occupatio...
1,2,\n Age: 53\n Gender: Female\n Occupat...
2,3,\n Age: 23\n Gender: Male\n Occupatio...
3,4,\n Age: 24\n Gender: Male\n Occupatio...
4,5,\n Age: 33\n Gender: Female\n Occupat...


## Defining Helper functions

In [ ]:
def get_last_10_ratings(user_id):
    """
    retrieve last 15 ratings according to the timestamp
    :param user_id: 
    :return: 
    """
    user_ratings = rating_df[rating_df['user_id'] == user_id].sort_values(by='timestamp', ascending=False).head(10)
    user_movies = movies_df[movies_df['movie_id'].isin(user_ratings['movie_id'])]
    return user_ratings.merge(user_movies, on='movie_id')

# --------------------------------------------------------------

def analyze_user_prompt(user_info, last_10_movies):
    """
    Generating the initial prompt to capture user's characteristics
    :param user_info: user information (age, gender, occupation)
    :param last_10_movies: dataframe
    :return: 
    """
    # getting rating summary
    ratings_summary = '\n'.join(
        f"Movie: {row['movie_info']}\nRating: {row['rating']}" for _, row in last_10_movies.iterrows()
    )
    
    # Generating prompt
    prompt = f"""
Consider below information about the user:
User Info:
{user_info}

User's Last 15 Movies and Ratings:
{ratings_summary}

Analyze the user's characteristics based on their history and provide a summary of the analysis. The analysis should include what type of movies the user enjoys and what type is not interesting for them. The output should be a JSON file where the key is "Analysis" and the value is the output
"""
    return prompt

# --------------------------------------------------------------

def rate_new_movies_prompt(user_info, analysis, test_movies):
    """
    Generate the proper prompt to ask the LLM to provide ratings for the recommendations 
    :param user_info: user information (text)
    :param analysis: LLM's analysis based on user's background (text)
    :param test_movies: testset
    :return: 
    """
    # test movie summaries
    movies_summary = '\n'.join(test_movies['movie_info'].tolist())
    
    prompt = f"""
Consider below information about a user
User Info:
{user_info}

Analysis:
{analysis}

Based on the user's background and user's characteristics, rate the following movies on behalf of the user:
{movies_summary}

Output should be a JSON array containing movie_id and rating.
"""
    return prompt

# --------------------------------------------------------------


def get_llm_response(prompt):
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-0125",
        prompt=prompt,
        max_tokens=500
    )
    client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)
    return response.choices[0].text.strip()
